In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from imgaug import augmenters as iaa
import imgaug as ia
import os
from filemanager import get_file_paths as gfp
import matplotlib.font_manager as fm

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
IMAGE_CHANNEL = 3
NUM_IMAGE = 10000

NUM_CLASS = 50

tf.random.set_seed(42)
ia.seed(42)

# 레이블 불러오기

In [ ]:
dataset = gfp.File_Control.searchAllFilesInDirectoryByDir("/raid/food_competition_KT_set1/train", "jpg")
dataset.sort()
label_dict = {}
for i in range(len(dataset)):
    label = os.path.dirname(dataset[i][0])
    label = os.path.basename(label)
    label_dict[str(i)] = label
print(label_dict)

# 기존 데이터 불러오기

In [ ]:
x = np.load('/raid/food_competition_KT_set1/kfood_img.npy')
y = np.load('/raid/food_competition_KT_set1/kfood_y.npy')

np.random.seed(0)
np.random.shuffle(x)
np.random.seed(0)
np.random.shuffle(y)

# Augmentation

In [ ]:
rand_aug = iaa.RandAugment(n=3, m=7)

def augment(images):
    # Input to `augment()` is a TensorFlow tensor which
    # is not supported by `imgaug`. This is why we first
    # convert it to its `numpy` variant.
    images = tf.cast(images, tf.uint8)
    return rand_aug(images=images.numpy())

# 모델 학습

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def resnet_model(img_height, img_width, num_classes):
    model = ResNet50V2(
        include_top=True,
        input_shape=(img_width, img_height, 3),
        weights=None,
        classes=50,
    )
    return model

model = resnet_model(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CLASS)
model.summary()

In [ ]:
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
prop = fm.FontProperties(fname=path, size=18)

In [ ]:
#dataset = read_dataset(100, './dataset', 'kt_food')

NUM_FOLD = 5
EPOCHS = 30
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE

kfold = KFold(n_splits=NUM_FOLD, shuffle=True, random_state=30)

fold_no = 1
results = []

for train_index, test_index in kfold.split(x):
    
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    x_train = augment(x_train)
    
    print("test index : ", test_index)
    print("test index shape : ", test_index.shape)
    
    model = resnet_model(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CLASS)
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    print('-' * 50)
    print(f'Training for fold {fold_no} ...')
    model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=EPOCHS)
    eval_results = model.evaluate(x_test, y_test)
    print("test loss, test acc : ", eval_results)
    results.append(eval_results)
    fold_no = fold_no + 1
    
    break
    
